# Peticiones con requests

Hasta ahora hemos visto como implementar una API con Python, pero ¿cómo interactuamos desde Python con una API? Pues generando peticiones HTTP directamente desde el código sin necesidad de un navegador. 

Para hacerlo tenemos a nuestra disposición un potente módulo externo llamado `requests`:

```bash
$ pip install requests
```

In [ ]:
!pip install requests

## Primera petición GET

El módulo `requests` permite utilizar diferentes métodos como `get` y `post`, hagamos una prueba:

In [ ]:
import requests

r = requests.get('https://www.wikipedia.es/')

print(type(r)) # tipo del dato

Podemos ver información de la respuesta analizando los atributos del objeto `Response`:

In [ ]:
print(r.text) # contenido de la respuesta

In [ ]:
print(r.status_code) # código de la respuesta

In [ ]:
print(r.headers) # cabeceras de la respuesta

In [ ]:
print(r.headers['date'])             # fecha de la petición
print(r.headers['last-modified'])    # fecha de última modificación
print(r.headers['content-type'])     # tipo del contenido
print(r.headers['server'])           # servidor
print(r.headers['content-language']) # idioma

## Parámetros en las peticiones

La herramienta `httpbin` es un servicio de peticiones y respuestas HTTP gratuito que proporciona un conjunto de rutas para pruebas. Podemos hacer una petición a `https://httpbin.org/get` y pasarle algunos datos para que los retorne de vuelta.

Los parámetros GET se envían en el formato clave-valor, en Python podemos definirlos en un diccionario que generalmente recibe el nombre de `payload` (datos de carga):

In [ ]:
url = 'https://httpbin.org/get'

payload = {
    'nombre': 'Hector Costa Guzman',
    'intereses':['Python', 'Videojuegos'],
    'edad': 33,
}

r = requests.get(url, params=payload)

print(r.text)

La respuesta es una cadena en formato `json` que deberíamos *parsear* para extraer los datos, algo que veremos en la próxima lección.

## Peticiones POST

Como ya hemos comentado las peticiones POST sirven para enviar datos recopilados en un formulario. Podemos hacer la misma petición que antes utilizando `post` y cambiando la ruta a `http://httpbin.org/post`. 

Para enviar los datos tendremos que cambiar el argumento `params` por `data` para especificar datos de formularios:

In [ ]:
url = 'https://httpbin.org/post'

payload = {
    'nombre': 'Hector Costa Guzman',
    'intereses':['Python', 'Videojuegos'],
    'edad': 33,
}

r = requests.post(url, data=payload)

print(r.text)

En caso de que un servidor solo acepte `payload` en formato JSON hay que utilizar el argumento `json=payload` en lugar de `data=payload`:

In [ ]:
url = 'https://httpbin.org/post'

payload = {
    'nombre': 'Hector Costa Guzman',
    'intereses':['Python', 'Videojuegos'],
    'edad': 33,
}

r = requests.post(url, json=payload)

print(r.text)

## Errores en las peticiones

Es posible que en algunas ocasiones los servidores HTTP no se encuentren activos o las peticiones devuelvan un error.

Vamos a pedir a `httpbin` que simule una respuesta de error haciendo una petición GET a `https://httpbin.org/status/404`:

In [ ]:
r = requests.get('https://httpbin.org/status/404')

Como veremos el código de respuesta es 404 (no encontrado):

In [ ]:
print(r.status_code)

Este error podemos tratarlo en un bloque `try-except` de la siguiente forma:

In [ ]:
import requests
from requests import HTTPError

try:
    # Hacemos la petición al sitio de pruebas
    r = requests.get("https://httpbin.org/status/404")
    # Pedimos que se invoque una excepcion HTTPError si hay algún fallo
    r.raise_for_status()
    # Si no hay un fallo mostramos algo de prueba
    print(r.status_code)
except HTTPError as ex:
    # Si se ha invocado una excepción HTTPError mostramos el resultado
    print(ex)

## Peticiones con timeout

Otra posibilidad es que en lugar de devolver un error el servidor simplemente no responda o tarde mucho tiempo.

Por defecto las peticiones no tienen un `timeout`, por lo que `requests` quedará en espera hasta que el servidor responda. Podemos simular esta situación estableciendo un tiempo de `delay` de 5 segundos en una petición a la ruta `https://httpbin.org/delay/5`:

In [ ]:
r = requests.get('https://httpbin.org/delay/5')

In [ ]:
print(r.status_code)

Sin embargo podemos indicarle un tiempo máximo para realizar la petición, por ejemplo si le indicamos 3 segundos fallará porque es menos que los 5 que tarda en responder:

In [ ]:
r = requests.get('https://httpbin.org/delay/5', timeout=3)

Como vemos se devuelve un error `TimeoutError`, debemos envolver el bloque de código en un `try-except` y tratar este caso de forma específica:

In [ ]:
from requests.exceptions import Timeout

try:
    requests.get('https://httpbin.org/delay/5', timeout=3)
except Timeout:
    print("ERROR: La petición ha tardado más de 3 segundos")

Las posibilidades del módulo `requests` son tantas como utilizar manualmente el navegador, os dejo la [documentación oficial](https://requests.readthedocs.io/en/latest/) para aprender más.